# This code may be dead, I can't get a good r2 no matter what I do

In [1]:
#Dependencies
import pandas as pd
import keras_tuner as kt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

### Getting Data for feeding the model

In [2]:
#Read in cleaned csv into a pandas dataframe
filename = "training_data.csv"
try:
    df = pd.read_csv(f"Data/{filename}")
except FileNotFoundError:
    raise FileNotFoundError(f"The file '{filename}' does not exist. Please make sure it is in the 'Data' folder.")

df = df.dropna()
df.head()

,Unnamed: 0,fips_code,year,niaaa_pop_21_plus,census_total_pop,census_percent_pop_21_plus,census_percent_pop_18_24,census_percent_pop_25_34,census_percent_pop_35_44,census_percent_pop_45_54,...,tpc_state_beer_tax_rates_dollar_gal,bea_personal_income_dollars,bea_disp_inc_dollars,bea_percapita_personal_income_dollars,bea_percapita_disp_inc_dollars,census_median_household_inc_dollars,niaaa_total_beer_vol_consumed_gallons,niaaa_legal_adult_per_capita_beer_consumed_gallons,ba_craft_beer_produced_gallons,ba_legal_adult_per_capita_craft_beer_produced_gallons
0,51,1,2011,3467668,4802740,0.722,0.101,0.127,0.128,0.143,...,1.05,167882.2,152835.4,34887,31760,41415,101246000,29.20,29202.0,0.01
1,52,2,2011,502699,722718,0.696,0.107,0.147,0.129,0.149,...,1.07,37870.6,34789.2,52415,48150,67825,14688320,29.22,4870317.0,9.69
2,53,4,2011,4574939,6482505,0.704,0.100,0.134,0.127,0.130,...,0.16,227203.3,206128.2,35244,31975,46709,132997688,29.07,2141480.0,0.47
3,54,5,2011,2103411,2937979,0.714,0.096,0.130,0.124,0.137,...,0.24,100449.5,91406.0,34180,31103,38758,51707608,24.58,126449.0,0.06
4,55,6,2011,26698281,37691912,0.708,0.106,0.143,0.137,0.139,...,0.20,1713428.0,1504645.4,45509,39963,57287,643901979,24.12,62627626.0,2.35


In [3]:
#Split data for testing and training
yColName = "ba_craft_beer_produced_gallons"
X = df.drop(yColName, axis=1).values
y = df[yColName].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
#Scale the data

#Create a StandardScaler instances
scaler = StandardScaler()

#Fit the StandardScaler
X_scaler = scaler.fit(X_train)

#Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Model Creation

In [6]:
#Create model function for keras tuner
regression = True
MAX_NEURONS = 128
MAX_LAYERS = 10

def create_model(hp):
    nnModel = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'linear'])

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, MAX_LAYERS)):
        nnModel.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=1, max_value=MAX_NEURONS, step=8), activation=activation))

    # Last layer for binary classification
    if not regression:  
        nnModel.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
        nnModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    elif regression:
        nnModel.add(tf.keras.layers.Dense(units=1, activation="linear"))
        nnModel.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

    return nnModel

In [7]:
#Use the tuner to generate the best model
tuner = kt.Hyperband(
    create_model,
    objective="mse",
    max_epochs=20,
    hyperband_iterations=2
)

tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))
nn = tuner.get_best_models(num_models=1)[0]

Trial 60 Complete [00h 00m 05s]
mse: 595490169159680.0

Best mse So Far: 110462616207360.0
Total elapsed time: 00h 03m 03s


In [8]:
#Create an EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

fit_model = nn.fit(X_train_scaled, y_train, epochs=500, callbacks=[early_stopping])

Epoch 1/500
12/12 [==============================] - 1s 2ms/step - loss: 107662171701248.0000 - mse: 107662171701248.0000
Epoch 2/500
12/12 [==============================] - 0s 2ms/step - loss: 108141891026944.0000 - mse: 108141891026944.0000
Epoch 3/500
12/12 [==============================] - 0s 2ms/step - loss: 105791184961536.0000 - mse: 105791184961536.0000
Epoch 4/500
12/12 [==============================] - 0s 2ms/step - loss: 106177438416896.0000 - mse: 106177438416896.0000
Epoch 5/500
12/12 [==============================] - 0s 2ms/step - loss: 103341837254656.0000 - mse: 103341837254656.0000
Epoch 6/500
12/12 [==============================] - 0s 2ms/step - loss: 103830104571904.0000 - mse: 103830104571904.0000
Epoch 7/500
12/12 [==============================] - 0s 2ms/step - loss: 105983133089792.0000 - mse: 105983133089792.0000
Epoch 8/500
12/12 [==============================] - 0s 2ms/step - loss: 103458002698240.0000 - mse: 103458002698240.0000
Epoch 9/500
12/12 [=====

In [9]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 146684155789312.0000 - mse: 146684155789312.0000 - 147ms/epoch - 37ms/step
Loss: 146684155789312.0, Accuracy: 146684155789312.0


In [10]:
from sklearn.metrics import r2_score

# Assuming nn is your trained model and X_test_scaled, y_test are your test data
predictions = nn.predict(X_test_scaled)

# Flatten predictions and ground truth for compatibility with r2_score
predictions = predictions.flatten()

# Calculate R-squared
r_squared = r2_score(y_test, predictions)

print(f'R-squared: {r_squared}')

4/4 [==============================] - 0s 2ms/step
R-squared: 0.7086324263888665


#### All cells below are previous outputs of the cell above as we go through the process of optimization, along with what we changed